# 6.12.1. Función de tensión de Airy en coordenadas polares

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Diego Andrés Alvarez, <daalvarez@unal.edu.co>  | Marzo 24, 2023      | Adaptando el código de MAXIMA a PYTHON |

## Cálculo de los esfuerzos en términos de la función de tensión de Airy en coordenadas polares

In [1]:
from sympy import init_printing, latex, sin, cos, symbols, diff, Function, Eq, solve, trigsimp, expand

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Se definen las variables y funciones simbólicas necesarias:

In [2]:
r, t        = symbols('r, theta')
sr, st, trt = symbols("sigma_r, sigma_θ, tau_rθ")
phi         = Function('phi')(r, t)

Se definen las primeras derivadas utilizando la regla de la cadena:

In [3]:
d_dx = lambda f: diff(f,r)*cos(t) - diff(f,t)*sin(t)/r
d_dy = lambda f: diff(f,r)*sin(t) + diff(f,t)*cos(t)/r
d_dz = lambda f: diff(f,z)

Se especifican los esfuerzos $\sigma_x$, $\sigma_y$ y $\tau_{xy}$ en función de $\sigma_r$, $\sigma_\theta$ y $\tau_{r\theta}$:
\begin{align}
\sigma_x  &= \sigma_r\cos^2\theta + \sigma_\theta\sin^2\theta - \tau_{r\theta}\sin 2\theta \\
\sigma_y  &= \sigma_r\sin^2\theta + \sigma_\theta\cos^2\theta + \tau_{r\theta}\sin 2\theta \\
\tau_{xy} &= \frac{1}{2}\left(\sigma_r - \sigma_\theta\right)\sin 2\theta + \tau_{r\theta}\cos 2\theta
\end{align}

In [4]:
sx  = sr*cos(t)**2 + st*sin(t)**2 - trt*sin(2*t)
sy  = sr*sin(t)**2 + st*cos(t)**2 + trt*sin(2*t)
txy = (sr - st)*sin(2*t)/2  + trt*cos(2*t)

Relaciones de Airy para los esfuerzos, ecuación (***) cuando no se tienen fuerzas másicas, esto es haciendo $V(x,y)=0$:
\begin{align}
\sigma_x(x,y)  & = \frac{\partial^2 \phi(x,y)}{\partial y^2} + V(x,y) \\
\sigma_y(x,y)  & = \frac{\partial^2 \phi(x,y)}{\partial x^2} + V(x,y) \\    
\tau_{xy}(x,y) & = -\frac{\partial^2 \phi(x,y)}{\partial x \partial y}.
\end{align}

In [5]:
eq1 = Eq(sx,   d_dy(d_dy(phi)))
eq2 = Eq(sy,   d_dx(d_dx(phi)))
eq3 = Eq(txy, -d_dx(d_dy(phi)))

Se calculan $\sigma_r$, $\sigma_\theta$ y $\tau_{r\theta}$ en términos de la función de tensión de Airy $\phi$:

In [6]:
sol = trigsimp(solve([eq1, eq2, eq3], [sr, st, trt]))
sr  = sol[sr].expand()
st  = sol[st]
trt = sol[trt].expand()

In [7]:
imprimir(r"\sigma_r(r, \theta) = ", sr)

<IPython.core.display.Math object>

In [8]:
imprimir(r"\sigma_\theta(r, \theta) = ", st)

<IPython.core.display.Math object>

In [9]:
imprimir(r"\tau_{r\theta}(r, \theta) = ", trt)

<IPython.core.display.Math object>

## Verificación de que la función de tensión de Airy satisface las ecuaciones de equilibrio

Se define el laplaciano en coordenadas polares:
\begin{equation}
     \nabla^2 \phi = \frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2} 
     = \frac{\partial^2 \phi}{\partial r^2} + \frac{1}{r}\frac{\partial \phi}{\partial r} + \frac{1}{r^2}\frac{\partial^2 \phi}{\partial \theta^2}
\end{equation}

In [10]:
lapl = lambda f : diff(f,r,2) + (1/r)*diff(f,r) + (1/r**2)*diff(f,t,2)

Se reemplaza en las ecuaciones diferenciales de equilibrio
\begin{align}
 \frac{\partial \sigma_r}{\partial r} + \frac{1}{r}\frac{\partial \tau_{r\theta}}{\partial \theta} + \frac{\sigma_r - \sigma_\theta}{r} + b_r &= 0 \\
 \frac{\partial \tau_{r\theta}}{\partial r} + \frac{1}{r}\frac{\partial \sigma_\theta}{\partial \theta}  + \frac{2 \tau_{r\theta}}{r} + b_\theta &= 0
\end{align}
asumiendo que las fuerzas másicas son nulas, esto es $b_r = b_\theta = 0$:

In [11]:
# Primera ecuación de equilibrio:
eq1 = expand(diff(sr,r) + (1/r)*diff(trt,t) + (sr - st)/r) # == 0
imprimir("", eq1, " = 0")

<IPython.core.display.Math object>

In [12]:
# Segunda ecuación de equilibrio:
eq2 = expand(diff(trt,r) + (1/r)*diff(st,t) + 2*trt/r) # == 0
imprimir("", eq2, " = 0")

<IPython.core.display.Math object>

Se reemplaza en la ecuación diferencial bidimensional de compatibilidad en términos de esfuerzos
\begin{equation}
\left(\frac{\partial^2}{\partial r^2} + \frac{1}{r}\frac{\partial}{\partial r} + \frac{1}{r^2}\frac{\partial^2}{\partial \theta^2}\right)\left(\sigma_r + \sigma_\theta\right) =K_1 \left(\frac{\partial b_r}{\partial r} + \frac{b_r}{r} + \frac{1}{r}\frac{\partial b_\theta}{\partial \theta}\right)
\end{equation}
asumiendo que las fuerzas másicas son nulas, esto es $b_r = b_\theta = 0$:

In [13]:
eq3 = expand(lapl(sr + st)) # == 0
imprimir("", eq3, " = 0")

<IPython.core.display.Math object>

Se calcula el biarmónico de la función de tensión de Airy en coordenadas polares:

In [14]:
biarm = expand(lapl(lapl(phi)))

imprimir(r"\nabla^4 \phi(r, \theta) = ", biarm)

<IPython.core.display.Math object>

Observe que la ecuación diferencial bidimensional de compatibilidad en términos de esfuerzos se reduce al biarmónico de $\phi$:

In [15]:
eq3 - biarm